Show Best so far of each Example image

In [ ]:
import sys
sys.path.append('./see-segment')

In [ ]:
def read_pop(filename):
    """Read Text output"""
    print(f"Reading in {filename}")
    inlist = []
    fitness = []
    with open(filename,'r') as myfile:
        for line in myfile:
            if (len(line) > 1):
                x,fit,pop = eval(line)
                inlist.append(pop)
                fitness.append(fit)
    return inlist, fitness

In [ ]:
from pathlib import Path
def getimages(folder='./see_segment/Image_data/Examples/'):
    #def pickimage(

    directory = Path(folder)

    allfiles = sorted(directory.glob('*'))

    filelist = []
    masklist = []
    progresslist = []
    for file in allfiles:
        if file.suffix ==".jpg" or file.suffix ==".jpeg" or file.suffix ==".JPEG" or file.suffix ==".png":
            if not "_GT" in file.name and not "best" in file.name:
                filelist.append(file)
                mask = directory.glob(f"{file.stem}_GT*")
                for m in mask:
                    masklist.append(m)
                progresslist.append(Path(f"{folder}/_{file.stem}.txt"))
    return filelist, masklist, progresslist

In [ ]:
directory = "./see-segment/Image_data/Examples/"
[images, masks, progresslist] = getimages(directory)

In [ ]:
import numpy as np
from see.Segmentors import segmentor
from see.ColorSpace import colorspace
from see.Workflow import workflow
from see.Segment_Fitness import segment_fitness
from see import base_classes 
from see.git_version import git_version
from see import GeneticSearch, Segmentors
import matplotlib.pylab as plt
from imageio import v3 as imageio

workflow.addalgos([colorspace, segmentor, segment_fitness])

fig = plt.figure(figsize=(20, 40))

rows = len(images)

row = 0
for imfile, gtfile, bestfile in zip(images, masks, progresslist):
    mydata = base_classes.pipedata()
    mydata.append(imageio.imread(imfile))
    mydata.gtruth = imageio.imread(gtfile) 
    
    my_ax = plt.subplot(rows, 3, (row*3)+1)
    my_ax.imshow(mydata[0])
    my_ax.axis('off')
    
    my_ax = plt.subplot(rows, 3, (row*3)+2)
    my_ax.imshow(mydata.gtruth)
    my_ax.axis('off')
    
    if bestfile.exists():
        if mydata[0].shape == mydata.gtruth.shape:
            [inlist, fitness] = read_pop(bestfile)
            best = inlist[np.argmin(fitness)]

            workflow.addalgos([colorspace, segmentor, segment_fitness])
            wf = workflow(paramlist=best)

            mydata = wf.pipe(mydata)

            my_ax = plt.subplot(rows, 3, (row*3)+3)
            my_ax.imshow(mydata[-1])
            my_ax.set_title(mydata.fitness)
            my_ax.axis('off')
    else:
        pass
        #print('no bestfile')
    row+=1